In [38]:
ENV["LINES"] = 30
ENV["COLUMNS"] = 300

300

In [3]:

using Pipe

In [4]:
function pz(x :: AbstractArray)
    println(typeof(x), ": ", size(x))
end

pz (generic function with 1 method)

In [5]:
using PyCall
@pyimport nltk

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2499: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [6]:
function tokenize(sentence::String)
    convert(Array{String,1},nltk.word_tokenize(sentence))
end

tokenize (generic function with 1 method)

In [7]:
include("load_embeddings.jl")

load_embeddings (generic function with 1 method)

In [8]:
LL,word_index,indexed_words =  load_embeddings("embeddings-scaled.EMBEDDING_SIZE=50.txt");
size(LL) |> println
word_index|> typeof |> println
indexed_words|> typeof |> println

(50,268810)
Dict{String,BitArray{1}}
Array{String,1}


In [9]:
typealias Embedding Vector{Float64}
typealias Embeddings Matrix{Float64}
typealias Words Union(AbstractArray{ASCIIString,1},AbstractArray{String,1})
type RAE
    L::Matrix{Float64}
    word_index::Dict{String,BitVector}
    indexed_words::Vector{String}
    
    W_e::Matrix{Float64}
    W_d::Matrix{Float64}
   
end


function RAE(L::Matrix{Float64}, word_index::Dict{String,BitVector}, indexed_words::Vector{String})
    emb_width = size(L,1)
    
    W_e =0.01*randn(emb_width,emb_width*2+1) 
    #W_d = 0.01*randn(emb_width*2,emb_width+1)
    W_d = [pinv(W_e)[1:end-1,:] zeros(size(pinv(W_e),1)-1) ] #Cheat (Actually why can't I always do this to initialize?);
    RAE(L,word_index, indexed_words, W_e, W_d)
end

function eval_embedding(rae::RAE, input::String)
    k= word_index[ii]
    rae.L*k
end

function eval_embeddings(rae::RAE, inputs::Words)
    ks= @pipe inputs |> map(ii -> word_index[ii], _) |> hcat(_...)
    rae.L*ks
end

function eval_embeddings(rae::RAE, c_is::Embeddings, c_js::Embeddings)
    @assert size(c_is)==size(c_js)
    bias_in = ones(1,size([c_is], 2))
    tanh(rae.W_e*[c_is;c_js;bias_in])
end



eval_embeddings (generic function with 2 methods)

In [10]:
function eval_scores(rae::RAE, c_is::Embeddings, c_js::Embeddings)
    pps=eval_embeddings(rae, c_is, c_js)
    c_ijs = [c_is;c_js]
    bias_in = ones(1,size(pps, 2))
    ĉ_ijs = tanh(rae.W_d*[pps;bias_in])
    
    sum((c_ijs-ĉ_ijs).^2,1)
    
end


eval_scores (generic function with 1 method)

In [11]:
function word_adjancy_matrix(sentence_len::Int)
    #TODO: Replace this whole adjancy matrix with a linked list
    [(ii==jj+1) || (ii==jj-1)  for ii in 1:sentence_len, jj in 1:sentence_len]
end


function eval_to_tree(rr::RAE,sentence::String)
    eval_to_tree(rr, tokenize(sentence))
end

function eval_to_tree(rr::RAE, sentence::Words)
    backlookup = sentence
    cs = eval_embeddings(rr, backlookup)
    AA = word_adjancy_matrix(size(cs,2))
    
    score_total = 0.0
    while(any(AA))
        iis, jjs = findn(AA)

        pps = eval_embeddings(rr, cs[:,iis],cs[:,jjs])
        scores = eval_scores(rr, cs[:,iis],cs[:,jjs])
        best_pair_ind = indmax(scores)
        

        ii_best = iis[best_pair_ind]
        jj_best = jjs[best_pair_ind]
        pp_best = pps[:,best_pair_ind]
        score_total+=scores[best_pair_ind]
        
        cs = [cs pp_best]
        backlookup = [backlookup, (backlookup[ii_best],backlookup[jj_best])]
        
        #Adjust Adjacency Matrix

        AA[ii_best,jj_best] = false
        AA[jj_best,ii_best] = false
        AA = [AA; AA[ii_best,:] | AA[jj_best,:]] # Add row
        AA = [AA AA[:,ii_best] | AA[:,jj_best]] # Add col
        AA[ii_best,:] = AA[jj_best,:] = false
        AA[:,ii_best] = AA[:,jj_best] = false #Remove anything that was adjacent to old

    end
    tree = backlookup[end]
    embedding = cs[:,end]
    tree, embedding, score_total
end


eval_to_tree (generic function with 2 methods)

In [12]:
rr = RAE(LL,word_index,indexed_words);

sent = "the boy destroyed the house"
sent_toks = tokenize(sent)

tree, pp, score_total = eval_to_tree(rr,sent_toks)


((("destroyed",("the","house")),("the","boy")),[-0.00514894,0.0111239,0.0154326,0.000781568,-0.00344358,-0.00135539,-0.0033673,-0.00655161,0.00804477,0.00249908  …  0.01198,-0.00587113,-0.0231239,-0.00756754,-0.00602867,-0.009554,-0.00748828,-0.00373229,-0.00161056,-0.00259715],12.65163093381063)

In [14]:
#tree data in tree is not use, other than it's structure.
#((("the","house"),("destroyed",("the","boy")))  is equivalent to ((("",""),("",("",""))) 

function reconstruct(rae::RAE, pp::Embedding)
    bias_in = 1
    ĉ_ijs = tanh(rae.W_d*[pp;bias_in])
    ĉ_is = ĉ_ijs[1:end/2]
    ĉ_js = ĉ_ijs[end/2+1:end]
    ĉ_is, ĉ_js
end

function unfold(rae::RAE, tree::(String,String), pp::Embedding)
    ĉ_is, ĉ_js = reconstruct(rae, pp)
    [ĉ_is ĉ_js]
end


function unfold(rae::RAE, tree::(Any,String), pp::Embedding)
    p̂_is, ĉ_js = reconstruct(rae, pp)
    ĉ_is = unfold(rae, tree[1], p̂_is)
    [ĉ_is ĉ_js]
end

function unfold(rae::RAE, tree::(String,Any), pp::Embedding)
    ĉ_is, p̂_js = reconstruct(rae, pp)
    ĉ_js = unfold(rae, tree[2], p̂_js)
    [ĉ_is ĉ_js]
    
end

function unfold(rae::RAE, tree::(Any,Any), pp::Embedding)
    p̂_is, p̂_js = reconstruct(rae, pp)
    ĉ_is = unfold(rae, tree[1], p̂_is)
    ĉ_js = unfold(rae, tree[2], p̂_js)
    [ĉ_is ĉ_js]
end

    

unfold (generic function with 4 methods)

In [44]:


tree, pp, score_total = eval_to_tree(rr,"1 1")
ĉs = unfold(rr,tree,pp)

show_bests(rr, ĉs)


20x4 Array{Any,2}:
 "1"              0.76309   "1"           0.70818 
 "4"              0.688818  "2"           0.682614
 "6"              0.664929  "7"           0.670441
 "3"              0.645222  "97"          0.659939
 "2"              0.642393  "3"           0.623277
 "detailed"       0.609192  "2.5"         0.613707
 "longer"         0.607712  "Newlands"    0.603756
 "non"            0.601455  "4"           0.60357 
 "72"             0.601402  "6"           0.599686
 "7"              0.599989  "8"           0.598776
 "supply-demand"  0.599945  "Subic"       0.590737
 "90-day"         0.594884  "5"           0.588673
 "cumulative"     0.593713  "\$"          0.577644
 "ADRs"           0.593608  "closing"     0.577303
 "9"              0.589779  "Oct"         0.576998
 "end-month"      0.587698  "Grandpuits"  0.576282
 "Eurograde"      0.587064  "Yorkshire"   0.574502
 "free-fall"      0.581038  "Target"      0.574046
 "401"            0.579868  "Rover"       0.572343
 "12"       

In [16]:
function cosine_dist(a,b)
    (a⋅b)/(norm(a)*norm(b))
end

function neighbour_dists(cc::Vector{Float64}, globe::Matrix{Float64})
    [cosine_dist(cc, globe[:,ii]) for ii in 1:size(globe,2)]
end


function show_best(rae::RAE,ĉ::Embedding, nbest=20)
    candidates=neighbour_dists(ĉ,rae.L)   
    best_cands = [ (findfirst(candidates,score), score)
                    for score in select(candidates,1:nbest, rev=true)[1:nbest]]
    vcat([[rae.indexed_words[ii] score] for (ii,score) in best_cands]...)
end

function show_bests(rae::RAE,ĉs::Embeddings, nbest=20)
    hcat([show_best(rae,ĉs[:,ii],nbest) for ii in 1:size(ĉs,2)]...)
end


show_bests (generic function with 2 methods)

In [161]:
function depth_inc(ele::(Int,String))
    (ele[1]+1,ele[2])
end

function unfold_struct(tree::(Any,Any))
    left_tree = unfold_struct(tree[1]) 
    left = @pipe left_tree |> map(depth_inc,_)
    right_tree = unfold_struct(tree[2]) 
    right = @pipe right_tree |> map(depth_inc,_)
    [left, right, (0,"")]
end

function unfold_struct(tree::(Any,String))
    left_tree = unfold_struct(tree[1]) 
    left = @pipe left_tree |> map(depth_inc,_)
    [left, (0,tree[2]), (0,"")]
end
function unfold_struct(tree::(String,Any))
    right_tree = unfold_struct(tree[2]) 
    right = @pipe right_tree |> map(depth_inc,_)
    [(0,tree[1]),right, (0,"")]
end
function unfold_struct(tree::(String,String))
    [(0,tree[1]), (0, tree[2]), (0,"")]
end

function print_tree(tree::(Any,Any))
    
    for (depth,word ) in unfold_struct(tree)
        println("\t"^depth, word)
    end
end

print_tree (generic function with 1 method)

In [149]:
print_tree(tree)

	the
	house
	
	destroyed
		the
		boy
		
	



In [156]:
aa=[1 2 3 4 5 6 7 8]
aa[end/4+1:end]

6-element Array{Int64,1}:
 3
 4
 5
 6
 7
 8

In [119]:
@pipe ("a", "b") |> join(_,"\n") 

a
b

In [133]:
tree

(("the","house"),("destroyed",("the","boy")))

In [20]:
import IJulia

In [24]:
help(IJulia.undisplay)

undisplay (generic function with 1 method)


300

In [36]:
aa = rand('a':'z', 100,100)

100x100 Array{Char,2}:
 'q'  's'  'e'  'c'  'x'  'o'  't'  'e'  'u'  'u'  'c'  't'  'k'  't'  'm'  'p'  'a'  'f'  'w'  'a'  'v'  'd'  'a'  'g'  'v'  'o'  'g'  'v'  'z'  'v'  …  'g'  'q'  'z'  'g'  't'  'h'  'y'  'u'  'h'  'o'  'l'  'z'  'l'  's'  's'  'o'  'q'  't'  'g'  'u'  'w'  'q'  'o'  'v'  'u'  'v'  'h'  'c'  'e'
 'q'  'w'  'b'  'x'  'u'  'q'  'b'  'k'  'j'  'g'  'w'  'c'  'c'  'x'  'w'  'x'  'o'  'h'  'u'  'f'  'e'  'm'  'g'  'z'  'y'  'i'  'x'  't'  'o'  'g'     'k'  'v'  'k'  'e'  'x'  't'  'd'  'h'  's'  'l'  'e'  'v'  'b'  'f'  'o'  'w'  'e'  'c'  'x'  'e'  'n'  'g'  'j'  'w'  'r'  'q'  'f'  'y'  'b'
 'h'  'f'  'v'  'x'  'h'  'e'  'm'  'h'  'q'  'a'  'g'  'n'  'z'  'n'  'a'  'e'  'o'  'i'  'f'  'a'  'h'  'w'  'e'  'z'  'q'  'e'  'u'  'p'  'p'  'p'     'e'  'o'  't'  'k'  'p'  'g'  'z'  'x'  'm'  'o'  'y'  'q'  'c'  'q'  'x'  'i'  'g'  'g'  'e'  'l'  'q'  'q'  'b'  'b'  'i'  'd'  't'  'n'  'u'
 'g'  'u'  'e'  'l'  'x'  'b'  'u'  'b'  'w'  'c'  't'  'f'  'q'  'h'  'd'  'o'  'q